In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)
import os
os.chdir('..')

In [2]:
from langchain_chinese import ChatZhipuAI, ZhipuAIEmbeddings
from langchain_chinese import LocalFilesLoader, create_qa_chain
from langchain_community.vectorstores import Chroma

docs = LocalFilesLoader()
chunks = docs.load_and_split()

db = Chroma.from_documents(chunks, ZhipuAIEmbeddings())

chain = create_qa_chain(ChatZhipuAI(), db.as_retriever())

In [3]:
chain.invoke("供应商达标的业绩要求是什么")

AIMessage(content='供应商达标的业绩要求包括：\n\n1. 月销售额：与我方合作的月份，经我方销售的产品月总价不得低于人民币3万元。\n2. 稳定的财务状况：供应商应具备健全和稳定的财务状况，以及良好的信用记录。\n3. 流动资金：供应商应具备足够的流动资金来支持合同执行。')

In [4]:
async for chunk in chain.astream_events("供应商达标的业绩要求是什么?", version="v1"):
	# print(chunk)
	# print(chunk['event'])
	if(chunk['event'] == "on_chat_model_stream"):
		print(chunk['data']['chunk'].content, end="|", flush=True)
	elif(chunk['event'] == "on_retriever_end"):
		for doc in chunk['data']['output']['documents']:
			print(doc.metadata['source'])


/Users/xuehongwei/Library/Caches/pypoetry/virtualenvs/langchain-chinese-gSQlHcwW-py3.9/lib/python3.9/site-packages/langchain_core/_api/beta_decorator.py:86: LangChainBetaWarning: This API is in beta and may change in the future.
  warn_beta(


/Users/xuehongwei/github/wencheng/data/documents/autogpt-demo/供应商资格要求.pdf
/Users/xuehongwei/github/wencheng/data/documents/autogpt-demo/供应商资格要求.pdf
/Users/xuehongwei/github/wencheng/data/documents/autogpt-demo/求职简历.docx
/Users/xuehongwei/github/wencheng/data/documents/autogpt-demo/供应商资格要求.pdf
供应商|达|标的|业绩|要求|包括|以下|几点|：|

1|.| 月|销售额|：|与我|方|合作的|月份|，|经|我|方|销售|的产品|月|总价|不得|低于|人民币|3|万元|。|
2|.| |稳定的|财务|状况|：|供应商|应|具备|健全|和|稳定的|财务|状况|，|以及|良好的|信用|记录|。|
3|.| |流动|资金|：|供应商|应|具备|足够的|流动|资金|来|支持|合同|执行|。||